### Задание
#### Необходимо провести анализ тональности сообщений на позитивные и негативные (бинарная классификация).
#### Данные:
#### train.csv – данные для тренировки. Столбцы text – само сообщение, label – метка (0 – негативное, 1 – позитивное).
#### test.csv – данные для тестирования. Столбцы index – номер сообщения, text – само сообщение.
#### Необходимо определить тональность (0 или 1) в файле train.csv (тут вероятно должно быть test.csv ?) и сохранить получившийся результат в формате prediction.csv c разделителем точка с запятой. Столбцы, которые должен содержать prediction.csv:
#### · index – номер сообщения
#### · label – предсказанная тональность

#### Качество полученного предсказания будет считаться по метрике F1.

In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics import f1_score
import stop_words
import warnings
warnings.filterwarnings('ignore')

In [2]:
# загрузка train датасета
df3_train = pd.read_csv("train.csv", low_memory=False)
df3_train.head()

,text,label
0,На нас обиделась Оля....Жизнь потеряла смысл. ...,0
1,RT @PolinaZuzy: http://t.co/ZYwbUowIti единств...,1
2,"@AHamueva Машину назвать малышкой,мать ты ебу ...",1
3,RT @malina23_: отличный день в школе!:*теперь ...,1
4,@Serebryakovaaa не то слово))) у него что ни м...,1


In [3]:
df3_train.info()
# пустых значений не обнаружено

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136100 entries, 0 to 136099
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    136100 non-null  object
 1   label   136100 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.1+ MB


In [4]:
# загрузка test датасета
df3_test = pd.read_csv("test.csv", low_memory=False)
df3_test.head()

,index,text
0,61947,"Блин, я только сейчас понял, что на день рожде..."
1,208586,"@kostarikara да ты все время бегаешь где-то,до..."
2,187819,У меня скоро на всех одноклассников компромат ...
3,5277,"Блин время 4 утра,а я залипаю в телефон((("
4,149042,На выходных подстригся и купил себе очки для р...


In [5]:
df3_test.info()
# пустых значений не обнаружено

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90734 entries, 0 to 90733
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   90734 non-null  int64 
 1   text    90734 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.4+ MB


In [6]:
# удаление пустых значений (на случай, если входные файлы изменятся)
df3_train = df3_train[['text', 'label']].dropna()
df3_test = df3_test.dropna()

In [7]:
# удаление дубликатов
df3_train = df3_train.drop_duplicates('text')

In [8]:
# применение Стемминга
stemmer = SnowballStemmer(language='russian')
stop_words_ru = stop_words.get_stop_words('russian')

def clear_txt(txt):
    """ 
    Функция для очистки текста.
    - переводит текст в нижний регистр;
    - удаляет латиницу, знаки препинания, символы и цифры;
    - применяет Стемминг.
    """
    txt = txt.lower()
    txt = re.sub('[/+_!@#$A-Za-z0-9\n.,:()""«»;-]', ' ', txt)
    new_txt = ''
    for t in txt.split(' '):
        if len(t) > 0:
            new_txt = new_txt + stemmer.stem(t) + ' '
    return new_txt[:-1]

Как вариант было - привести к начальной форме в нижнем регистре, но посчитал лишним на данном этапе (сильно возрастёт по времени).

In [9]:
# разделение train датасета на 2 выборки - для обучения, и для тестирования качества работы модели
X_train, X_test, y_train, y_test = train_test_split(df3_train['text'], df3_train['label'], test_size=0.3)

In [10]:
# Передача в Vectorizer в качестве параметров: функция и стоп-слова.
# Преобразование слов в числовые данные, с определением "важности" слов

stop_words_ru = stop_words.get_stop_words('russian')
df_vectorizer = TfidfVectorizer(max_features=10000,
                                preprocessor=clear_txt,
                                stop_words=stop_words_ru)

X_train = df_vectorizer.fit_transform(X_train)

In [11]:
# обучение логистической регрессии и предсказание на выделенной тестовой выборке
#(CV версия и RandomForestClassifier давали примерно такой же результат 0.69-0.71)

lr = LogisticRegression(random_state=0, n_jobs=4, C=0.8)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(df_vectorizer.transform(X_test))

In [12]:
# оценка f1_score на выделенной тестовой выборке
f1_score(y_test, y_pred_lr)

0.7174474723346171

In [19]:
# предсказание на тестовом датасете, с трансформацией входных данных через Vectorizer
df3_test['label'] = lr.predict(df_vectorizer.transform(df3_test['text']))

In [20]:
df3_test.head(3)

,index,text,label
0,61947,"Блин, я только сейчас понял, что на день рожде...",0
1,208586,"@kostarikara да ты все время бегаешь где-то,до...",0
2,187819,У меня скоро на всех одноклассников компромат ...,1


In [21]:
# выделение необходимых столбцов и выгрузка в файл с указанными параметрами
df3_test = df3_test[['index', 'label']]
df3_test.to_csv("prediction.csv", sep=';', index=False)